## 

Autor: **Radosław Dąbkowski**

Numer indeksu: **325683**

Grupa: **1**
Numer: **2**

### Wstęp



In [143]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec  3 14:08:04 2022

@author: mgrzy
"""

import numpy as np
import gpxpy
import scipy
import gpxpy.gpx
import pandas as pd

def gpx2df(gpx_file_name, new_file_name = 'route_df.csv'):
    with open(gpx_file_name, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    route_info = []
    
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'time':point.time
                })
    
    route_df = pd.DataFrame(route_info)
    
    save_df(route_df, new_file_name)

def blh2xyz(fi, lam, h) -> np.array:
    '''
    Parameters
    ----------
    fi : float
        latitude [rad].
    lam : float
        longitude [rad].
    h : float
        height [m].
    Returns
    -------
    xyz : numpy array
        x, y, z [m].
    '''
    a = 6378137.0
    e2 = 0.00669438002290
    N = a/np.sqrt(1-e2*np.sin(fi)**2)
    x = (N+h)*np.cos(fi)*np.cos(lam)
    y = (N+h)*np.cos(fi)*np.sin(lam)
    z = (N*(1-e2)+h)*np.sin(fi)
    xyz = np.array([x, y, z])
    return xyz

def save_df(route_df, new_file_name= 'route_df.csv'):
    route_df.to_csv(new_file_name, index=False)
    print(f'file saved to {new_file_name}')
def read_route_df(file):
    route_df = pd.read_csv(file)
    return route_df

In [144]:
gpx2df('Afternoon_Ride.gpx')
df = read_route_df('route_df.csv')
df.head()

file saved to route_df.csv


,latitude,longitude,elevation,time
0,53.208030,23.326793,136.4,2022-07-28 13:26:13+00:00
1,53.208035,23.326818,136.4,2022-07-28 13:26:14+00:00
2,53.208038,23.326861,136.4,2022-07-28 13:26:16+00:00
3,53.208047,23.326925,136.3,2022-07-28 13:26:17+00:00
4,53.208052,23.326948,136.3,2022-07-28 13:26:18+00:00


In [145]:
import folium

m = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=12)

l = list(zip(df.latitude.values, df.longitude.values))

folium.PolyLine(l, color="red", weight=2.5, opacity=1).add_to(m)

# add points that show the elevation

for i in range(0, len(df), 100):
    folium.Circle(
        location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
        popup=df.iloc[i]['elevation'],
        radius=10,
        color='crimson',
        fill=True,
        fill_color='crimson'
    ).add_to(m)

phis = df['latitude'].values
lambdas = df['longitude'].values
dists = []
for i in range(len(phis)-1):
    phi1 = phis[i]
    lam1 = lambdas[i]
    phi2 = phis[i+1]
    lam2 = lambdas[i+1]

    xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
    xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
    dist = np.linalg.norm(xyz1-xyz2)
    dists.append(dist)

# append the last distance
phi1 = phis[-1]
lam1 = lambdas[-1]
phi2 = phis[0]
lam2 = lambdas[0]

xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
dist = np.linalg.norm(xyz1-xyz2)
dists.append(dist)

df['distance'] = np.cumsum(dists)

# sample data
# 52.81 14.81      36.68
# 52.81 14.82      36.67
# 52.81 14.83      36.66
# 52.81 14.84      36.65
# 52.81 14.85      36.63
# 52.81 14.86      36.62
# 52.81 14.87      36.61
# 52.81 14.88      36.59
# 52.81 14.89      36.58
# 52.81 14.90      36.57
# 52.81 14.91      36.55
# 52.81 14.92      36.54
# 52.81 14.93      36.53
# 52.81 14.94      36.51
# 52.81 14.95      36.50
model = np.genfromtxt('siatka_gugik-geoid2011-PL-KRON86-NH.txt', skip_header=1)
model = model[~np.isnan(model[:, 2])]
# sort by phi and lam
model = model[model[:, 0].argsort()]
model = model[model[:, 1].argsort(kind='mergesort')]
# print the middle row
print(model[len(model)//2-1])
print(model[len(model)//2])
print(model[len(model)//2+1])

def interpolate_geoid(model, phi, lam):
    '''
    Parameters
    ----------
    model : numpy array
        geoid model.
    phi : float
        latitude [deg].
    lam : float
        longitude [deg].
    Returns
    -------
    h : float
        geoid height [m].
    '''
    # find the 4 closest points
    p1 = (model[:, 0] <= phi).nonzero()[0][-1] #wtf
    p2 = (model[:, 0] > phi).nonzero()[0][0]
    p3 = (model[:, 1] <= lam).nonzero()[0][-1]
    p4 = (model[:, 1] > lam).nonzero()[0][0]
    # get the 4 closest points
    phi1 = model[p1, 0]
    phi2 = model[p2, 0]
    lam1 = model[p3, 1]
    lam2 = model[p4, 1]
    h1 = model[p1, 2]
    h2 = model[p2, 2]
    h3 = model[p3, 2]
    h4 = model[p4, 2]
    # calculate the weights
    w1 = (phi2-phi)/(phi2-phi1)*(lam2-lam)/(lam2-lam1)
    w2 = (phi2-phi)/(phi2-phi1)*(lam-lam1)/(lam2-lam1)
    w3 = (phi-phi1)/(phi2-phi1)*(lam2-lam)/(lam2-lam1)
    w4 = (phi-phi1)/(phi2-phi1)*(lam-lam1)/(lam2-lam1)
    # interpolate
    h = w1*h1+w2*h2+w3*h3+w4*h4
    return h


# interpolate geoid
for phi, lam in zip(phis, lambdas):
    h1 = interpolate_geoid(model, phi, lam)
    print(f'phi: {phi}, lam: {lam}, h1: {h1}')
# show model
model

[54.27 19.48 28.92]
[54.28 19.48 28.9 ]
[54.29 19.48 28.87]
phi: 53.20803, lam: 23.326793, h1: 30.924699581320997
phi: 53.208035, lam: 23.326818, h1: 30.937658658685663
phi: 53.208038, lam: 23.326861, h1: 30.959955075987143
phi: 53.208047, lam: 23.326925, h1: 30.993134857607785
phi: 53.208052, lam: 23.326948, h1: 31.00505665960933
phi: 53.208065, lam: 23.327002, h1: 31.033045373421228
phi: 53.208084, lam: 23.327062, h1: 31.064138229063616
phi: 53.208101, lam: 23.327113, h1: 31.090565999362845
phi: 53.208127, lam: 23.327186, h1: 31.12839173796871
phi: 53.208161, lam: 23.327289, h1: 31.181765265003783
phi: 53.208202, lam: 23.327412, h1: 31.245501540696747
phi: 53.208258, lam: 23.327584, h1: 31.33462937341606
phi: 53.208287, lam: 23.327668, h1: 31.378154409217437
phi: 53.208316, lam: 23.32775, h1: 31.420641976491623
phi: 53.208344, lam: 23.327832, h1: 31.463130540652724
phi: 53.208372, lam: 23.327914, h1: 31.505618832638987
phi: 53.208401, lam: 23.327995, h1: 31.547586979105688
phi: 53.20

array([[52.84, 14.05, 37.73],
       [52.82, 14.06, 37.76],
       [52.83, 14.06, 37.73],
       ...,
       [50.87, 24.2 , 29.32],
       [50.88, 24.2 , 29.31],
       [50.89, 24.2 , 29.29]])